Да, это boilerplate код с документации pytorch, но helloworld он helloworld и есть. 

За счет одного скрытого слоя с 30 нейронами точность даже выше получилась чем в оригинальном примере.

То же самое, но с нуляhttps://github.com/30be/haskell-neural-net

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [12]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [16]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 30),
            nn.ReLU(),
            nn.Linear(30, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Using cpu device


In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.278712  [   64/60000]
loss: 2.276873  [ 6464/60000]
loss: 2.322223  [12864/60000]
loss: 2.251767  [19264/60000]
loss: 2.244335  [25664/60000]
loss: 2.225034  [32064/60000]
loss: 2.193209  [38464/60000]
loss: 2.192393  [44864/60000]
loss: 2.165960  [51264/60000]
loss: 2.174521  [57664/60000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 2.134141 

Epoch 2
-------------------------------
loss: 2.110574  [   64/60000]
loss: 2.072886  [ 6464/60000]
loss: 2.147185  [12864/60000]
loss: 2.027511  [19264/60000]
loss: 2.060933  [25664/60000]
loss: 2.019542  [32064/60000]
loss: 1.950438  [38464/60000]
loss: 1.987581  [44864/60000]
loss: 1.936858  [51264/60000]
loss: 1.921753  [57664/60000]
Test Error: 
 Accuracy: 61.1%, Avg loss: 1.889462 

Epoch 3
-------------------------------
loss: 1.885735  [   64/60000]
loss: 1.798685  [ 6464/60000]
loss: 1.894795  [12864/60000]
loss: 1.749874  [19264/60000]
loss: 1.806736  [25664/60000]
loss: 1.755433  [32064/600

In [18]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 79.7%, Avg loss: 1.075267 

